# Heuristic Algorithm Test

In this experiment we evaluate whether the coordinates we have annotated via graph layouting are usable to inform heuristic search algorithms.

The goals of this experiment are twofold:
- Identify, which heuristic algorithm performs best with the annotated coordinates.
- Identify, which method of coordinate annotation works best.

The notebook contains multiple experiments with the following purposes:
- Experiment 1: Identify good scale factors for the generated coordinates to give them a fair chance in the following experiment
- Experiment 2: Test algorithms on all coordinate annotations to identify the best coordinate annotations method.
- Experiment 3: Test all algorithms on the best coordinate annotation method to identify the best algorithm.

In [1]:
# correct working directory.
# This is necessary for imports because the notebook is not in the main folder of the project. 
if not "working_directory_corrected" in vars():
    %cd ..
    working_directory_corrected = True


import pandas as pd

from evaluation.timed_experiment import Timed_Experiment
from algorithms.dijkstra import Dijkstra
from algorithms.a_star import A_Star
from algorithms.heuristic_search import Heuristic_Search


# load dataset
from data.dataset import Dataset

c:\Users\frank\Documents\Teaching\LU\Planning and Optimization LU - Material\Planning Example Project\planning_example_project


c:\Users\frank\Documents\Teaching\LU\Planning and Optimization LU - Material\Planning Example Project\planning_example_project\environment\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Dataset
This experiment is based on data generated with the script generate_coordinates. This script applies all algorithms identified in coordinate_annotation.ipynb to the graph and generates coordinates. It does so 5 times since in previous experiments we noted that the coordinates differ significantly from each other from run to run.


While executing this script, we had to exclude some of the algorithms due to the following reasons:
- kamada_kawai and mds ran into memory issues and could not be executed
- davidson_harel and graphopt have not delivered a result within an hour of time. To keep the experiment manageable time-wise, they have been excluded based on this.


After these exclusions, the following algorithms remained: auto, drl, fruchterman_reingold



## Experiment 1 - Scale factor

As discussed in coordinate_annotation.ipynb, some coordinates where extremely close together. To give A* a fair chance we would like to rectify this by scaling coordinates so that they are in similar maximal distances.

**Procedure**

The cell below loads the first coordinate set of each algorithm and prints the minimum and maximum x and y coordinates. From this we will determine scale factors as potenties of 10 to assure that both coordinates are at least 10000 apart.

In [2]:
from asyncio.windows_events import INFINITE


for algorithm_name in ["auto", "drl", "fruchterman_reingold"]:
    file_name = f"if_{algorithm_name}_1"

    # load and convert graph
    dataset = Dataset()
    graph = dataset.load_graph()
    dataset.convert_to_spatial(graph,file_name)

    # find min and max of coordinates
    min_x = min(graph.node_positions, key= lambda coord: coord[0])[0]
    max_x = max(graph.node_positions, key= lambda coord: coord[0])[0]
    min_y = min(graph.node_positions, key= lambda coord: coord[1])[1]
    max_y = max(graph.node_positions, key= lambda coord: coord[1])[1]

    print("\n" + algorithm_name)
    print(f"min_x: {min_x}, max_x: {max_x}, Distance: {max_x - min_x}")		
    print(f"min_y: {min_y}, max_y: {max_y}, Distance: {max_y - min_y}")	

    


auto
min_x: -979.9690551757812, max_x: 974.4048461914062, Distance: 1954.3739013671875
min_y: -955.8806762695312, max_y: 950.1741333007812, Distance: 1906.0548095703125

drl
min_x: -962.9459228515625, max_x: 959.2142944335938, Distance: 1922.1602172851562
min_y: -979.453125, max_y: 961.8577270507812, Distance: 1941.3108520507812

fruchterman_reingold
min_x: -258.30335937319484, max_x: 290.7462575501768, Distance: 549.0496169233716
min_y: -276.3667291666116, max_y: 282.34564149108724, Distance: 558.7123706576988


**Results:**
The following distances have been calculated:

| Algorithm | X distance | Y Distance |
| --- | --- | --- |
| auto | 1954 | 1906 |
| drl | 1922 | 1941 |
| fruchterman_reingold | 549 | 558 |

**Conclusions:**

From this experiment it seems like the the coordinates are already quite far apart and don't need to be changed much.

We will apply the following scale factors:
- auto: 10
- drl: 10
- fruchterman_reingold: 100


## Experiment 2: Which coordinates are best?

The goal of this experiment is to determine which coordinates perform best with A* and Dijkstra.

**Procedure**

In the below cell we test all heuristic search methods with all coordinate generation variants. 

The following algorithms are tested:
- Dijkstra's algorithm
- A*
- Heuristic search.
Edge lengths and heuristics have been calculated as manhattan distance of the node coordinates. This distance has been used because it is computationally more efficient than euclidean distance.


Each algorithm will be tested on all 5 variants of the three layout algorithms. To have a comparison benchmark we also run them without coordinates.

Each run will be comprised of 100 planning problems, all with the same seed.From each experiment we will collect the following data:
- *average_time*: The average time they took to solve each problem.
- *nr_extended*: The number of nodes that were extended.
- *time per extension*: The average time required for one extension.

After running the experiments we will use these values to discuss the results. 
We will use the average time to choose which coordinate generation variant to use for further experiments.

In [3]:

algorithm_names = [None, "auto", "drl", "fruchterman_reingold"]
scale_factors = [0, 10,10,100]
planners = [Dijkstra, A_Star, Heuristic_Search] 

for index in range(len(algorithm_names)):
    algorithm_name = algorithm_names[index]	
    scale_factor = scale_factors[index]	
    
    for variation in range(5):
        file_name = f"if_{algorithm_name}_{variation}"
        dataset = Dataset()
        graph = dataset.load_graph()
        if algorithm_name is not None:
            dataset.convert_to_spatial(graph,file_name, scale_factor=scale_factor)

        for planner in planners:

            print(f"\nrunning {planner} on algorithm {algorithm_name} with variation {variation}")
            experiment = Timed_Experiment(graph, planner, 100, random_seed=42, verbose = False)
            experiment.run()
            print("Average Time: ", int(experiment.get_average_time()), "ns")
            print("Nr Extended: ", experiment.get_nr_extensions())
            print("Time per extension: ", int(experiment.get_average_extension_time()), "ns")



running <class 'algorithms.dijkstra.Dijkstra'> on algorithm None with variation 0
Average Time:  347109831 ns
Nr Extended:  3388818
Time per extension:  10242 ns

running <class 'algorithms.a_star.A_Star'> on algorithm None with variation 0
Average Time:  374697343 ns
Nr Extended:  3388818
Time per extension:  11056 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm None with variation 0
Average Time:  155182488 ns
Nr Extended:  3290487
Time per extension:  4716 ns

running <class 'algorithms.dijkstra.Dijkstra'> on algorithm None with variation 1
Average Time:  347854034 ns
Nr Extended:  3388818
Time per extension:  10264 ns

running <class 'algorithms.a_star.A_Star'> on algorithm None with variation 1
Average Time:  355511803 ns
Nr Extended:  3388818
Time per extension:  10490 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm None with variation 1
Average Time:  144160165 ns
Nr Extended:  3290487
Time per extension:  4381 ns


**Results**

The following table collects the average time over all experiments. Since the runs without layout algorithms are identical we will only pick one representation from them. All times are in ns.

| Experiment | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Benchmark 4| 319 ms | 365 ms | 164 ms |
| auto 0 | 301 ms | 321 ms | 240 ms | 
| auto 1 | 312 ms | 330 ms | 238 ms |
| auto 2 | 315 ms | 319 ms | 215 ms |
| auto 3 | 274 ms | 253 ms | 175 ms |
| auto 4 | 234 ms | 243 ms | 176 ms |
| drl 0 | 233 ms | 243 ms | 168 ms |
| drl 1 | 231 ms | 241 ms | 166 ms |
| drl 2 | 229 ms | 236 ms | 165 ms |
| drl 3 | **220 ms** | **234** ms | **163 ms** |
| drl 4 | 235 ms | 243 ms | 166 ms |
| fruchterman_reingold 0 | 259 ms | 272 ms | 173 ms |
| fruchterman_reingold 1 | 275 ms | 284 ms | 172 ms |
| fruchterman_reingold 2 | 278 ms | 283 ms | 168 ms |
| fruchterman_reingold 3 | 270 ms | 279 ms | 172 ms |
| fruchterman_reingold 4 | 280 ms | 272 ms | 173 ms |

The following table collects the average number of extended nodes over all experiments. Since the runs without layout algorithms are identical we will only pick one representation from them. 


| Experiment | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Benchmark 4| 3388818 | 3388818 | 3290487 |
| auto 0 | 3218843 | 2842061 | 3256738 |
| auto 1 | 3274321 | 2852468 | 3311445 |
| auto 2 | 3293419 | 2858840 | 3202355 |
| auto 3 | 3274679 | 2875837 | 3216756 |
| auto 4 | 3324951 | 2897464 | 3236311 |
| drl 0 | 3288192 | 2872806 | 3144215 |
| drl 1 | 3301889 | 2853738 | 3155147 |
| drl 2 | 3257531 | 2845389 | 3106382 |
| drl 3 | 3259777 | 2870461 | 3173221 |
| drl 4 | 3367594 | 2869428 | 3241371 |
| fruchterman_reingold 0 | 3243333 | 2850068 | 3253287 |
| fruchterman_reingold 1 | 3293176 | 2859680 | 3211560 |
| fruchterman_reingold 2 | 3349097 | 2873033 | 3259015 |
| fruchterman_reingold 3 | 3353094 | 2877601 | 3250718 |
| fruchterman_reingold 4 | 3310525 | 2876480 | 3253627 |

**Discussion**

In the table above, we can see different behavious for the three algorithms.
For Dijkstra's Algorithm and A*, all layouting algorithms show a clear improvement over the benchmark. For both "drl" is the most beneficial in terms of runtime. Heuristic Search, on the other hand, rarely shows an improvement when applied to the layouting algorithms. Of the three layouting algorithms, "drl" also shows the lowest times.

Regarding the number of extended nodes, we can see that A* is the one that profits most from the heuristic. Heuristic Search and Dijkstra's algorithm only show minor reductions in this number or even increases, in the case of heuristic search.

**Conclusion**
Variant 3 of drl has the lowest times for all three algorithms. We will thus continue with this variant.


## Experiment 3: Which algorithm is best?

The goal of this experiment is to evaluate which of the three algorithms is best for the tested coordinate annotation method.

**Experiment Setup**

Based on the previous experiment we have selected the coordinate generation algorithm TODO in variation TODO.
We will run A*, Dijkstra and Heuristic search, as described in the previous experiment, on these coordinates to determine which algorithm performs best. 

Each run will be comprised of 1000 planning problems, all with the same seed. From each experiment we will collect the following data:
- *average_time*: The average time they took to solve each problem.
- *nr_extended*: The number of nodes that were extended.
- *time per extension*: The average time required for one extension.
We will collect this data for the dataset as a whole but also for only sucessful / failed runs.

While only the overall average time will be used for selecting which algorithm to test further, we will use the other data points for discussion about the differences between algorithm performance.


In [6]:

algorithm_name = "drl"
scale_factor = 10
variation = 3
planners = [Dijkstra, A_Star, Heuristic_Search] 


file_name = f"if_{algorithm_name}_{variation}"
dataset = Dataset()
graph = dataset.load_graph()
dataset.convert_to_spatial(graph,file_name, scale_factor=scale_factor)

for planner in planners:
    print(f"\nrunning {planner} on algorithm {algorithm_name} with variation {variation}")
    experiment = Timed_Experiment(graph, planner, 1000, random_seed=42, verbose = False)
    experiment.run()
    print("Average Time (All): ", int(experiment.get_average_time()), "ns")
    print("Nr Extended (All): ", experiment.get_nr_extensions())
    print("Time per extension (All): ", int(experiment.get_average_extension_time()), "ns")

    print("Successes:", len(experiment.successful_extensions))
    print("Average Time (Success):", int(experiment.get_average_successful_time()) , "ns")
    print("Nr Extended (Success): ", experiment.get_nr_successful_extensions())
    print("Time per extension (Success): ", int(experiment.get_average_successful_extension_time()), "ns")

    print("Fails:", len(experiment.unsuccesful_extensions))
    print("Average Time (Fail):", int(experiment.get_average_unsuccessful_time()) , "ns")
    print("Nr Extended (Fail): ", experiment.get_nr_unsuccessful_extensions())
    print("Time per extension (Fail): ", int(experiment.get_average_unsuccessful_extension_time()), "ns")



running <class 'algorithms.dijkstra.Dijkstra'> on algorithm drl with variation 3
Average Time (All):  238575305 ns
Nr Extended (All):  33779826
Time per extension (All):  7062 ns
Successes: 151
Average Time (Success): 199035790 ns
Nr Extended (Success):  4080023
Time per extension (Success):  7366 ns
Fails: 849
Average Time (Fail): 245607657 ns
Nr Extended (Fail):  29699803
Time per extension (Fail):  7020 ns

running <class 'algorithms.a_star.A_Star'> on algorithm drl with variation 3
Average Time (All):  254185415 ns
Nr Extended (All):  30265211
Time per extension (All):  8398 ns
Successes: 151
Average Time (Success): 44480279 ns
Nr Extended (Success):  565408
Time per extension (Success):  11879 ns
Fails: 849
Average Time (Fail): 291482794 ns
Nr Extended (Fail):  29699803
Time per extension (Fail):  8332 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm drl with variation 3
Average Time (All):  171803938 ns
Nr Extended (All):  32913709
Time per extensi

**Results**

| Value | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Average Time (All)| 238 ms | 254 ms | 171 ms |
| Average Time (Success)| 199 ms | 44 ms  | 121 ms |
| Average Time (Fail)| 245 ms  | 291 ms | 180 ms |
| Nr Extended (All)| 33.779.826 | 30.265.211 | 32.913.709 |
| Nr Extended (Success)| 4.080.023 | 565.408 | 3.213.811 |
| Nr Extended (Fail)| 29.699.803 | 29.699.803 | 29.699.898 |
| Time per extension (All)| 7062 ns | 8398 ns | 5219 ns |
| Time per extension (Success)| 7366 ns | 11879 ns | 5716 |
| Time per extension (Fail)| 7020  | 8332 ns | 5166 |


**Discussion**

Overall, the fastest Algorithm on this dataset is heuristic search. We will use this algorithm in future experiments.

In addition to this general conclusion, we can derive a few more insights from the experiment:
- In all three algorithms, the average time for sucessful runs where significantly lower than those for failed runs. On the one hand, this can be attributed to the fact that in failed runs the entire search space has to be covered while sucessful runs terminate earlier. O the other hand, we can see differences in the number of states extended by the two algorithms that use the heuristic directly(A* and Heuristic search) when compared to Dijkstra, which only uses edge lengths. This indicates, that the heuristic is helpful in directing the search in some way.
- A* is the algorithm that profits the most from the heuristic. The difference in average times between sucessful and failed runs is the by far the biggest of the three. The reason that A* does not come out best overall is likely twofold:
   - A* has the highest time per extension due to the algorithmic complexity of keeping a priority queue of nodes and calculating the heuristci. 
   Due to the overall low number of successful cases (abouut 15% in this experiment), the advantage of A* does not come into play often enough for it to matter in the aveage time.
- Heuristic search is fastest mostly due to it's low per extension time. While it extends more states than A* (though less than Dijkstra's Algorithm), it takes a lot less time per state.
